In [4]:
import numpy as np
from numbers import Number
from pyscf import fci
from copy import deepcopy, copy

#define vacuum
#take in operator string
#format: string of indices, bitstring of creation vs annihilation
#return normal-ordered operator string plus contracted terms, according to Wick's theorem



In [5]:
norbs = 3
Nocc = 2
vacuumOccupations = [1 for i in range(Nocc)] + [0 for i in range (norbs - Nocc)]
print(vacuumOccupations)

[1, 1, 0]


In [6]:
class basicOperator:
    def __init__(self, orbital_, creation_annihilation_, spin_):
        self.orbital = orbital_ #orbital index
        self.spin = spin_ #1 for alpha, 0 for beta
        self.creation_annihilation = creation_annihilation_ #1 for creation, 0 for annihilation
        self.quasi_cre_ann = self.creation_annihilation

    def applyFermiVacuum(self, vacuum):
        self.quasi_cre_ann = (vacuum[self.orbital] and (not self.creation_annihilation)) or (not vacuum[self.orbital] and self.creation_annihilation)

    def __str__(self):
        string = "a"
        if bool(self.creation_annihilation):
            string = string + "^"
        else:
            string = string + "_"
        if bool(self.spin):
            string = string + "{" + str(self.orbital) + "\\alpha}"
        else:
            string = string + "{" + str(self.orbital) + "\\beta}"
        return string

    def __eq__(self, other):
        if isinstance(other, basicOperator):
            return self.orbital == other.orbital and self.spin == other.spin and self.creation_annihilation == other.creation_annihilation
        return False

    def conjugate(self):
        return basicOperator(self.orbital, not self.creation_annihilation, self.spin)

    def apply(self, state):
        neleca, nelecb = state.nelec[0], state.nelec[1]
        norb = state.norb
        if bool(self.creation_annihilation):
            if bool(self.spin):
#                if neleca == norb:
#                    state.array = np.array([[0.]])
#                    state.nelec = (0, 0)
#                else:
                state.array = fci.addons.cre_a(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca + 1, nelecb)
            else:
#                if nelecb == norb:
#                    state.array = np.array([[0.]])
#                    state.nelec = (0, 0)
#                else:
                state.array = fci.addons.cre_b(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca, nelecb + 1)
        else:
            if bool(self.spin):
#                if neleca == 0:
#                    state.array = np.array([[0.]])
#                    state.nelec = (0, 0)
#                else:
                state.array = fci.addons.des_a(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca - 1, nelecb)
            else:
#                if nelecb == 0:
#                    state.array = np.array([[0.]])
#                    state.nelec = (0, 0)
#                else:
                state.array = fci.addons.des_b(state.array, state.norb, state.nelec, self.orbital)
                state.nelec = (neleca, nelecb - 1)

In [7]:
class operatorProduct:
    def __init__(self, operatorList_, prefactor_=1.):
        self.operatorList = operatorList_
        self.prefactor = prefactor_

    def __str__(self):
        string = str(self.prefactor)
        if(len(self.operatorList) > 0):
            string = string + " * "
        for o in self.operatorList:
            string = string + o.__str__()
        return string

    def __mul__(self, other):
        if isinstance(other, operatorProduct):
            return operatorProduct(self.operatorList + other.operatorList, self.prefactor * other.prefactor)
        elif isinstance(other, operatorSum):
            newSummandList = []
            for s in other.summandList:
                newSummandList.append(self * s)
            return operatorSum(newSummandList)
        elif isinstance(other, Number):
            return operatorProduct(self.operatorList, self.prefactor * other)

    def __rmul__(self, other):
        if isinstance(other, Number):
            return operatorProduct(self.operatorList, other * self.prefactor)

    def __add__(self, other):
        if isinstance(other, operatorProduct):
            return operatorSum([self, other])
        elif isinstance(other, operatorSum):
            return operatorSum([self, other.summandList])

    def __eq__(self, other):
        if isinstance(other, operatorProduct):
            return self.operatorList == other.operatorList and self.prefactor == other.prefactor

    def applyFermiVacuum(self, vacuum):
        for o in self.operatorList:
            o.applyFermiVacuum(vacuum)
            
    def checkNilpotency(self):
        nonZero = True
        i = 0
        while i < len(self.operatorList):
            j = i + 1
            while j < len(self.operatorList):
                if self.operatorList[j] == self.operatorList[i]:
                    nonZero = False
                elif self.operatorList[j] == self.operatorList[i].conjugate():
                    break
                j = j + 1
            i = i + 1
        return int(nonZero)

    def conjugate(self):
        return operatorProduct([o.conjugate() for o in self.operatorList], np.conjugate(self.prefactor))

    def apply(self, state):
        i = len(self.operatorList)
        while i>0:
            i = i - 1
            self.operatorList[i].apply(state)

In [8]:
class operatorSum:
    def __init__(self, summandList_):
        self.summandList = summandList_

    def __str__(self):
        string = self.summandList[0].__str__()
        s = 1
        while s < len(self.summandList):
            string = string + " + " + self.summandList[s].__str__()
            s = s + 1
        return string

    def __add__(self, other):
        if isinstance(other, operatorSum):
            return operatorSum(self.summandList + other.summandList)
        elif isinstance(other, operatorProduct):
            if other.prefactor == 0:
                return self
            return operatorSum(self.summandList + [other])
            
    def __radd__(self, other):
        if isinstance(other, operatorProduct):
            if other.prefactor == 0:
                return self
            return operatorSum([other] + self.summandList)

    def __mul__(self, other):
        if isinstance(other, operatorProduct):
            newSummandList = []
            for s in self.summandList:
                newSummandList.append(s * other)
            return operatorSum(newSummandList)
        elif isinstance(other, operatorSum):
            newSummandList = []
            for o in other.summandList:
                partialSum = self * o
                newSummandList = newSummandList + partialSum.summandList
            return operatorSum(newSummandList)
        elif isinstance(other, Number):
            return operatorSum([self.summandList[s] * other for s in range(len(self.summandList))])

    def __rmul__(self, other):
        if isinstance(other, operatorProduct):
            newSummandList = []
            for s in self.summandList:
                newSummandList.append(other * s)
            return operatorSum(newSummandList)
        elif isinstance(other, Number):
            return operatorSum([other * self.summandList[s] for s in range(len(self.summandList))])

    def apply(self, state_):
        result = state(np.array([[0.]]), state_.norb, state_.nelec)
        for o in self.summandList:
            statecopy = deepcopy(state_)
            o.apply(statecopy)
            print(statecopy.array)
            result = statecopy + result
            print(result.array)
#        state1, state2 = copy(state), copy(state)
#        self.operator1.apply(state1)
#        self.operator2.apply(state2)
#        state = state1 + state2
        state_ = deepcopy(result)
        print(state_.array)

In [9]:
def normalOrder(operator, vacuum):
    '''
    Input: an operatorProduct or operatorSum and a list corresponding to which orbitals are occupied in the Fermi vacuum
    Output: normal ordered form of input, with respect to vacuum
    '''
    if isinstance(operator, operatorSum):
        return operatorSum([normalOrder(product, vacuum) for product in operator.summandList])
    quasiCreationList, quasiAnnihilationList = [], []
    quasiCreationCount = 0
    sign = 1
    for o in range(len(operator.operatorList)):
        op = operator.operatorList[o]
#        print(operator.orbital)
#        print(operator.creation_annihilation)
#        print(operator.spin)
        op.applyFermiVacuum(vacuum)
        if bool(op.quasi_cre_ann):
            quasiCreationList.append(op)
            if (o - quasiCreationCount) % 2 == 1:
                sign = -sign
            quasiCreationCount += 1
        else:
            quasiAnnihilationList.append(op)
    return operatorProduct(quasiCreationList + quasiAnnihilationList, sign * operator.prefactor)

In [10]:
oP = operatorProduct([basicOperator(1,1,1), basicOperator(1,0,1)])
nOP = normalOrder(oP, vacuumOccupations)
for i in range(len(nOP.operatorList)):
    print(nOP.operatorList[i])

print(oP)
print(nOP)

print((nOP + oP) * (nOP + oP))

a = basicOperator(1,1,1)
b = basicOperator(1,1,1)

print(a==b)

print((nOP * nOP).checkNilpotency())

a_{1\alpha}
a^{1\alpha}
1.0 * a^{1\alpha}a_{1\alpha}
-1.0 * a_{1\alpha}a^{1\alpha}
1.0 * a_{1\alpha}a^{1\alpha}a_{1\alpha}a^{1\alpha} + -1.0 * a^{1\alpha}a_{1\alpha}a_{1\alpha}a^{1\alpha} + -1.0 * a_{1\alpha}a^{1\alpha}a^{1\alpha}a_{1\alpha} + 1.0 * a^{1\alpha}a_{1\alpha}a^{1\alpha}a_{1\alpha}
True
1


In [11]:
def anticommute(operatorProduct_, first):
    '''
    Apply fermionic anticommutation relation to two adjacent second-quantized operators in an operatorProduct
    '''
    operatorList_ = operatorProduct_.operatorList
#    firstIndex = operatorList_[first].orbital
#    secondIndex = operatorList_[first + 1].orbital
    commutedTerm = operatorProduct(operatorList_[:first] + operatorList_[first + 1] + operatorList_[first] + operatorList_[first + 1:], operatorProduct_.prefactor * (-1))
#    if operatorList_[first].quasi_cre_ann == operatorList_[second].quasi_cre_ann:
    return commutedTerm
#    elif firstIndex == secondIndex:
#        contractedTerm = operatorProduct(operatorList_[:first] + operatorList_[first+2:], operatorProduct_.prefactor)
#        return operatorSum([contractedTerm, commutedTerm])
#    else:
#        return commutedTerm

In [12]:
def anticommuteInPlace(operatorProduct_, first):
    '''
    Apply fermionic anticommutation relation in place to two adjacent second-quantized operators in an operatorProduct
    '''
    operatorList_ = operatorProduct_.operatorList
    if operatorProduct_ == []:
        return
    firstOperatorCopy = deepcopy(operatorList_[first])
    operatorList_[first], operatorList_[first + 1] = operatorList_[first + 1], firstOperatorCopy
    operatorProduct_.prefactor = -operatorProduct_.prefactor
    return

In [13]:
def contract(operatorProduct_, first, second):
    '''
    Make a contraction between the indices of operators at the positions given by first and second in operatorProduct_
    '''
    operatorList_ = operatorProduct_.operatorList
    if operatorList_ == []:
        return operatorProduct_
    firstIndex = operatorList_[first].orbital
    secondIndex = operatorList_[second].orbital
    if firstIndex == secondIndex:
        if not bool(operatorList_[first].quasi_cre_ann) and bool(operatorList_[second].quasi_cre_ann):
            return operatorProduct(operatorList_[:first] + operatorList_[first+1:second] + operatorList_[second + 1:], operatorProduct_.prefactor * ((-1) ** (1 + second - first)))
    return operatorProduct([],0)

In [14]:
def reorderForContraction(operatorProduct_, first, second):
    '''
    Reorder operators using anticommutation relation so that two operators to be contracted are in positions 0 and 1
    '''
    operatorProductCopy = deepcopy(operatorProduct_)
    i = first
    j = second
    while i > 0:
        anticommuteInPlace(operatorProductCopy, i-1)
        i = i - 1
    while j > 1:
        anticommuteInPlace(operatorProductCopy, j-1)
        j = j - 1
    return operatorProductCopy


In [15]:
def reorderForMultipleContraction(operatorProduct_, pairsList):
    '''
    Reorder operators using anticommutation relation so that each pair of operators to be contracted are adjacent
    '''
    operatorProductCopy = deepcopy(operatorProduct_)
    finalPosition = 0
    for positionPair in pairsList:
        i = positionPair[0]
        j = positionPair[1]
        while i > finalPosition:
            anticommuteInPlace(operatorProductCopy, i-1)
            i = i - 1
        while j > finalPosition + 1:
            anticommuteInPlace(operatorProductCopy, j-1)
            j = j - 1
        finalPosition = finalPosition + 2
    return operatorProductCopy

In [16]:
def updateFlattenedPositionList(flattenedPositions):
    for i in range(len(flattenedPositions)):
        if flattenedPositions[i] > flattenedPositions[1]:
            flattenedPositions[i] -= 2
        elif flattenedPositions[i] > flattenedPositions[0]:
            flattenedPositions[i] -= 1
    return flattenedPositions[2:]

In [17]:
def multipleContraction(operatorProduct_, positionPairsList):
    '''
    Contractions between multiple pairs of operators in a product, at the positions given by entries 2i and 2i+1 in positionPairsList
    '''
    product = deepcopy(operatorProduct_)
    while len(positionPairsList) > 0:
#    for positionPair in pairsList:
        reorderedProduct = reorderForContraction(product, positionPairsList[0], positionPairsList[1])
        product = contract(reorderedProduct, 0, 1)
        positionPairsList = updateFlattenedPositionList(positionPairsList) #NOTE: reduces length of positionPairsList by 2
    return product

In [18]:
def nFoldContractionFromStart(operatorProduct_, n):
    product = deepcopy(operatorProduct_)
    if (2 * n <= len(product.operatorList)):
        for i in range(n):
            product = contract(product, 0, 1)
    return product

In [19]:
vacuumOccupations2 = [0,0,0]
oP2 = operatorProduct([basicOperator(0,0,1), basicOperator(1,0,1), basicOperator(0,1,1), basicOperator(1,1,1)])
nOP2 = normalOrder(oP2, vacuumOccupations2)
print(oP2)
print(nOP2)

print("----")

print(contract(oP2, 0,2))
print(contract(oP2, 1,3))
print(contract(contract(oP2, 0,2),0,1))
print(nFoldContractionFromStart(oP2, 1))
print(len(oP2.operatorList))

print("----")

print(reorderForContraction(oP2,0,2))
print(nFoldContractionFromStart(reorderForContraction(oP2,0,2), 1))

print("----")

print(multipleContraction(oP2, []))

1.0 * a_{0\alpha}a_{1\alpha}a^{0\alpha}a^{1\alpha}
1.0 * a^{0\alpha}a^{1\alpha}a_{0\alpha}a_{1\alpha}
----
-1.0 * a_{1\alpha}a^{1\alpha}
-1.0 * a_{0\alpha}a^{0\alpha}
-1.0
0
4
----
-1.0 * a_{0\alpha}a^{0\alpha}a_{1\alpha}a^{1\alpha}
-1.0 * a_{1\alpha}a^{1\alpha}
----
1.0 * a_{0\alpha}a_{1\alpha}a^{0\alpha}a^{1\alpha}


In [20]:
positionPairs = [(0,2),(1,3)]
flattenedPositions = [pos for pair in positionPairs for pos in pair]
print(flattenedPositions)
flattenedPositionsAfterContraction = deepcopy(flattenedPositions)
for i in range(len(flattenedPositionsAfterContraction)):
    if flattenedPositionsAfterContraction[i] > flattenedPositionsAfterContraction[1]:
        flattenedPositionsAfterContraction[i] -= 2
    elif flattenedPositionsAfterContraction[i] > flattenedPositionsAfterContraction[0]:
        flattenedPositionsAfterContraction[i] -= 1
print(flattenedPositionsAfterContraction[2:])
print(flattenedPositions)

[0, 2, 1, 3]
[0, 1]
[0, 2, 1, 3]


In [21]:
def genPairOrderedLists(list_):
    '''
    Given an ordered list with an even number of elements, returns list of all permutations of that list where each adjacent pair is ordered, and where the first elements of pairs are ordered
    '''
    pairOrderedLists = []
    if len(list_) == 2:
        pairOrderedLists.append(list_)
    else:
        for i in range(1,len(list_)):
            swappedList = [list_[0]] + [list_[i]] + list_[1:i]
            if i < len(list_):
                swappedList = swappedList + list_[i+1:]
            subLists = genPairOrderedLists(swappedList[2:])
            for l in range(len(subLists)):
                pairOrderedLists.append(swappedList[:2] + subLists[l])
    return pairOrderedLists

In [22]:
objectList = ["a", "b", "c", "d", "e", "f"]
print(genPairOrderedLists(objectList))

[['a', 'b', 'c', 'd', 'e', 'f'], ['a', 'b', 'c', 'e', 'd', 'f'], ['a', 'b', 'c', 'f', 'd', 'e'], ['a', 'c', 'b', 'd', 'e', 'f'], ['a', 'c', 'b', 'e', 'd', 'f'], ['a', 'c', 'b', 'f', 'd', 'e'], ['a', 'd', 'b', 'c', 'e', 'f'], ['a', 'd', 'b', 'e', 'c', 'f'], ['a', 'd', 'b', 'f', 'c', 'e'], ['a', 'e', 'b', 'c', 'd', 'f'], ['a', 'e', 'b', 'd', 'c', 'f'], ['a', 'e', 'b', 'f', 'c', 'd'], ['a', 'f', 'b', 'c', 'd', 'e'], ['a', 'f', 'b', 'd', 'c', 'e'], ['a', 'f', 'b', 'e', 'c', 'd']]


In [23]:
from itertools import combinations

In [24]:
def getPositionsForMultipleContraction(operatorProduct_, n):
    nOperators = len(operatorProduct_.operatorList)
    operatorPositions = range(nOperators)
    nToChoose = 2 * n
    if nToChoose <= nOperators:
        return combinations(operatorPositions, nToChoose)


In [25]:
print(oP2)
chosenPositions = getPositionsForMultipleContraction(oP2, 2)
print([list(c) for c in chosenPositions])
print([genPairOrderedLists(list(d)) for d in chosenPositions])

1.0 * a_{0\alpha}a_{1\alpha}a^{0\alpha}a^{1\alpha}
[[0, 1, 2, 3]]
[]


In [26]:
def sumNFoldContractions(operatorProduct_, n):
    chosenPositions = getPositionsForMultipleContraction(operatorProduct_, n)
    if n == 0:
        return operatorProduct_
    operatorSum_ = operatorSum([])
    for c in chosenPositions:
        pairOrderedList = genPairOrderedLists(list(c))
        for l in pairOrderedList:
            operatorSum_ = operatorSum_ + multipleContraction(operatorProduct_, l)
    return operatorSum_

In [27]:
print(oP2)
print(multipleContraction(oP2, [0,2,1,3]))
print(sumNFoldContractions(oP2, 1))

1.0 * a_{0\alpha}a_{1\alpha}a^{0\alpha}a^{1\alpha}
-1.0
-1.0 * a_{1\alpha}a^{1\alpha} + -1.0 * a_{0\alpha}a^{0\alpha}


In [28]:
def wickExpand(operator, vacuum):
    if isinstance(operator, operatorSum):
        wickExpansion = operatorSum([], 0.)
        for product in operator.summandList:
            wickExpansion = wickExpansion + wickExpand(product, vacuum)
        return wickExpansion
    wickExpansion = operatorSum([normalOrder(operator, vacuum)])
    highestOrder = len(operator.operatorList) // 2
    for n in range(highestOrder):
        wickExpansion = wickExpansion + sumNFoldContractions(operator, n + 1)
    return normalOrder(wickExpansion, vacuum)

In [29]:
def vacuumExpectationValue(operator, vacuum):
    wickExpansion = wickExpand(operator, vacuum)
    vEV = 0.
    for summand in wickExpansion.summandList:
        if summand.operatorList == []:
            vEV =+ summand.prefactor
    return vEV

In [30]:
print(oP2)
print(vacuumOccupations)
print(wickExpand(oP2, vacuumOccupations))
print(vacuumExpectationValue(oP2, vacuumOccupations))

1.0 * a_{0\alpha}a_{1\alpha}a^{0\alpha}a^{1\alpha}
[1, 1, 0]
1.0 * a_{0\alpha}a_{1\alpha}a^{0\alpha}a^{1\alpha}
0.0


In [31]:
#Particle number-conserving excitation operators, normal ordered with respect to true vacuum
class excitation(operatorProduct):
    def __init__(self, creationList_, annihilationList_, spinList_):
        self.operatorList = []
        for i in range(len(creationList_)):
            self.operatorList.append(basicOperator(creationList_[i], 1, spinList_[i]))
        for i in range(len(annihilationList_)):
            self.operatorList.append(basicOperator(annihilationList_[i], 0, spinList_[i+len(creationList_)]))

In [68]:
class spinFreeExcitation(operatorSum):
    def __init__(self, creationList_, annihilationList_):
        self.summandList = []
        spinLists = np.reshape(np.zeros(len(creationList_)), (1, -1))
        for i in range(len(creationList_)):
            newspinLists = deepcopy(spinLists)
            for s in range(len(spinLists)):
                newspinLists[s,i] = 1
            spinLists = np.concatenate((spinLists, newspinLists))
        for l in range(len(spinLists)):
            spinList = spinLists[l]
            self.summandList.append(excitation(creationList_, annihilationList_, [*spinList, *spinList[::-1]]))

In [69]:
class operatorString:
    def __init__(self, string_):
        self.string = string_
        #list of indices in the string of second-quantised operators
        self.iS = []
        #bitstring of same length, with 1 corresponding to creation and 0 for annihilation
        self.cOA = []
        #bitstring of same length, with 1 corresponding to alpha and 0 for beta
        self.aOB = []

    #Get indices, alpha or beta, and creation or annihilation from string
    def parse(self):
        c = 0
        while c < len(self.string) - 1:
            self.iS.append(int(self.string[c]))
            if self.string[c+1] == "a":
                    self.aOB.append(1)
            elif self.string[c+1] == "b":
                    self.aOB.append(0)
            if c < len(self.string) - 2 and self.string[c+2] == "+":
                self.cOA.append(1)
                c = c + 3
            else:
                self.cOA.append(0)
                c = c + 2

    #Change string based on new indices, alpha or beta, and creation or annihilation
    def updateString(self):
        self.string = ""
        i = 0
        while i < len(self.iS):
            if bool(self.cOA[i]):
                if bool(self.aOB[i]):
                    self.string = self.string + str(self.iS[i]) + "a+"
                else:
                    self.string = self.string + str(self.iS[i]) + "b+"
            else:
                if bool(self.aOB[i]):
                    self.string = self.string + str(self.iS[i]) + "a"
                else:
                    self.string = self.string + str(self.iS[i]) + "b"
            i = i + 1

    #Apply the string of second-quantised operators to an arbitrary state object
    #Note that nelec does not yet account for the state being completely annihilated
    def apply(self, state):
        i = len(self.iS)
        while(i > 0):
            i = i - 1
#            print(i)
#            print(self.iS[i])
#            print(self.cOA[i])
#            print(self.aOB[i])
            neleca, nelecb = state.nelec[0], state.nelec[1]
            if bool(self.cOA[i]):
                if bool(self.aOB[i]):
                    state.array = fci.addons.cre_a(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca + 1, nelecb)
                else:
                    state.array = fci.addons.cre_b(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca, nelecb + 1)
            else:
                if bool(self.aOB[i]):
                    state.array = fci.addons.des_a(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca - 1, nelecb)
                else:
                    state.array = fci.addons.des_b(state.array, state.norb, state.nelec, self.iS[i])
                    state.nelec = (neleca, nelecb - 1)

In [70]:
class state:
    def __init__(self, array_, norb_, nelec_):
        self.array = array_
        self.norb = norb_
        self.nelec = nelec_

    def __add__(self, other):
        if (other.array == 0.).all():
            return self
        elif (self.array == 0.).all():
            return other
        elif self.norb == other.norb and self.nelec == other.nelec:
            newArray = self.array + other.array
            return state(newArray, self.norb, self.nelec)
        else:
            print("Adding states of different orbital or particle numbers or different spin projections is not supported. Returning original state")
            return self

    def __radd__(self, other):
        if other == 0:
            return self
        else:
            return self.__add__(other)


In [71]:
ref = state(np.array([[1.,0.,0.],[0.,0.,0.],[0.,0.,0.]]), 3, (2,1))

In [72]:
new = deepcopy(ref)

In [73]:
des0a, des0b = basicOperator(0,0,1), basicOperator(0,0,0)
des1a, des1b = basicOperator(1,0,1), basicOperator(1,0,0)
des2a, des2b = basicOperator(2,0,1), basicOperator(2,0,0)

cre0a, cre0b = basicOperator(0,1,1), basicOperator(0,1,0)
cre1a, cre1b = basicOperator(1,1,1), basicOperator(1,1,0)
cre2a, cre2b = basicOperator(2,1,1), basicOperator(2,1,0)

In [74]:
exc12a = operatorProduct([cre2a, des1a])
exc12b = operatorProduct([cre2b, des1b])
exc12 = operatorSum([exc12a, exc12b])

In [75]:
new.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [76]:
exc12.apply(new)

[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]
[[0. 0. 0.]
 [1. 0. 0.]
 [0. 0. 0.]]


In [77]:
new.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [78]:
new.nelec

(2, 1)

In [79]:
opStr = operatorString("0a1b+2a+")
opStr.parse()

In [80]:
opStr.iS

[0, 1, 2]

In [81]:
new = deepcopy(ref)
opStr.apply(new)

In [82]:
ref.nelec

(2, 1)

In [83]:
new.nelec

(2, 2)

In [84]:
new.array

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-1.,  0.,  0.]])

In [85]:
opStr1 = operatorString("0a+0b+1a+")
opStr1.parse()
print(opStr1.string)
print(opStr1.iS)
print(opStr1.aOB)
print(opStr1.cOA)

0a+0b+1a+
[0, 0, 1]
[1, 0, 1]
[1, 1, 1]


In [86]:
opStr2 = copy(opStr1)

In [87]:
type(opStr2.string)

str

In [88]:
opStr2.iS[2] = 2
print(opStr2.string)
print(opStr2.iS)
print(opStr2.aOB)
print(opStr2.cOA)

0a+0b+1a+
[0, 0, 2]
[1, 0, 1]
[1, 1, 1]


In [89]:
opStr2.updateString()

In [90]:
opStr2.string

'0a+0b+2a+'

In [91]:
ref.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [92]:
new = copy(ref)

In [93]:
new.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [94]:
opStr.apply(new)

In [95]:
new.array

array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [-1.,  0.,  0.]])

In [96]:
cre0a = basicOperator(0,1,1)

In [97]:
cre0a.apply(new)

In [98]:
new.array

array([[-1.,  0.,  0.]])

In [99]:
new.nelec

(3, 2)

In [100]:
des1 = operatorSum([basicOperator(1,1,0), basicOperator(1,0,0)])

In [101]:
des1.apply(new)

[[0.]]
[[0.]]
[[1. 0. 0.]]
[[1. 0. 0.]]
[[1. 0. 0.]]


In [102]:
E12 = spinFreeExcitation([2],[1])

In [103]:
E12.apply(new)

[[ 0. -1. -0.]]
[[ 0. -1. -0.]]
[[0. 0. 0.]]
[[ 0. -1. -0.]]
[[ 0. -1. -0.]]


In [104]:
new.array

array([[-1.,  0.,  0.]])

In [105]:
des1a.apply(new)

In [106]:
new.nelec

(2, 2)

In [107]:
new.array

array([[ 0.,  0.,  0.],
       [ 1., -0., -0.],
       [ 0.,  0.,  0.]])

In [108]:
ref.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [109]:
ref.nelec

(2, 1)

In [110]:
new = copy(ref)

In [111]:
des1b.apply(new)

In [112]:
new.array

array([[0.],
       [0.],
       [0.]])

In [113]:
new.nelec

(2, 0)

In [114]:
type(new)

__main__.state

In [115]:
ref.array

array([[1., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [117]:
def delta(i, j):
    return int(i == j)

In [132]:
def contract2operators(o1, o2):
    if o1.quasi_cre_ann:
        return 0
    elif o2.quasi_cre_ann and (o1.spin == o2.spin):
        return int(o1.orbital == o2.orbital)
    else:
        return 0

In [120]:
def recursiveFullContraction(operatorProduct_, vacuum):
    operatorProduct_.applyFermiVacuum(vacuum)
    operatorList_ = operatorProduct_.operatorList
    if len(operatorList_) == 0:
        result = operatorProduct_.prefactor
    elif len(operatorList_) == 2:
        result = operatorProduct_.prefactor * contract2operators(operatorList_[0], operatorList_[1])
    elif len(operatorList_) % 2 == 0:
        result = 0
        for i in range(1, len(operatorList_) - 1):
            if contract2operators(operatorList_[0], operatorList_[i]):
                result += pow(-1, i-1) * recursiveFullContraction(operatorProduct(operatorList_[1:i] + operatorList_[i+1:]))
            else:
                result += 0
        if contract2operators(operatorList_[0], operatorList_[-1]):
            result += recursiveFullContraction(operatorProduct(operatorList_[1:-1]))
        else:
            result += 0
    return result
        


In [135]:
print(oP2)
print(vacuumOccupations)
print(wickExpand(oP2, vacuumOccupations))
print(vacuumExpectationValue(oP2, vacuumOccupations))

1.0 * a_{0\alpha}a_{1\alpha}a^{0\alpha}a^{1\alpha}
[1, 1, 0]
1.0 * a_{0\alpha}a_{1\alpha}a^{0\alpha}a^{1\alpha}
0.0


In [136]:
print(recursiveFullContraction(oP2, vacuumOccupations))

0
